<a href="https://colab.research.google.com/github/jaynetra/AIForHealthCare_Mimic3/blob/main/Mimic3Visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade chart_studio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 2.3 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import scipy.optimize

import chart_studio
import chart_studio.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
import plotly.express as px
import plotly.io as pio

from google.colab import data_table
data_table.enable_dataframe_formatter()


In [3]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')


Authenticated


In [4]:
project_id = 'mimic3visualization'
from google.cloud import bigquery
client = bigquery.Client(project=project_id)

In [5]:
# Bar chart by gender
gender_df = client.query(''' select gender, count(gender) as count from `physionet-data.mimiciii_clinical.patients` group by gender''').to_dataframe()
fig = px.bar(gender_df, x='gender', y='count', title='Gender Distribution')
fig.show()

In [6]:
# Pie chart by marital status
ms = client.query(''' select marital_status, count(marital_status) as count from `physionet-data.mimiciii_clinical.admissions` group by marital_status''').to_dataframe()
fig = px.pie(ms, values='count', names='marital_status', title='Marital Status Distribution')
fig.show()


In [7]:
# find the top medications used by the patient, from prescription table group by the drug

prescriptions_df = client.query(''' select DRUG_NAME_GENERIC, count(DRUG_NAME_GENERIC) as count  from `physionet-data.mimiciii_clinical.prescriptions` group by DRUG_NAME_GENERIC limit 10''').to_dataframe()
# plot
fig = px.bar(prescriptions_df, x='DRUG_NAME_GENERIC', y='count', title='Top Medications')
fig.show()



In [8]:
#join diagnoses_icd to d_icd_diagnosis and group by icd9_code
diagnoses_df = client.query(
    ''' select icd9_code, count(icd9_code) as count from `physionet-data.mimiciii_clinical.diagnoses_icd` group by icd9_code limit 10''').to_dataframe()
# plot
fig = px.bar(diagnoses_df, x='icd9_code', y='count', title='Top Diseases')
fig.show()



In [9]:
# Plot length of stay
icu_df = client.query(''' select subject_id, HADM_ID, LOS from `physionet-data.mimiciii_clinical.icustays` ''').to_dataframe()
# find media Length of staty and plot histogram
fig = px.histogram(icu_df, x='LOS', title='Length of Stay Distribution')
fig.show()






In [10]:
query = \
"""
 select * from `physionet-data.mimiciii_demo.diagnoses_icd` a
 join `physionet-data.mimiciii_demo.d_icd_diagnoses` b on a.icd9_code = b.icd9_code

"""

df = client.query(query).to_dataframe()



In [11]:
df1 = df.groupby('LONG_TITLE')['LONG_TITLE'].count()
df2=df1.sort_values(ascending=False).head(10)
fig = px.bar(df2, x=df2.index, y=df2.values, title='Top Diseases')
fig.show()

In [12]:
!pip install DataProfiler

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 735.5/735.5 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 kB 6.4 MB/s eta 0:00:00
  Created wheel for HLL: filename=HLL-2.3.0-cp311-cp311-linux_x86_64.whl size=32972 sha256=ab639f5370f2fb485c5c52a340351e55f801f8c5d5453f373031e8468f27396d
  Stored in directory: /root/.cache/pip/wheels/ab/5a/b5/65fdce434afc0cd43e14484d244c3928c7f1a7943dd2ab1a2c
Successfully built HLL


In [18]:
# use profiler to profile patient.csv that is loaded in local drive for bonous point
# collab code to access google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [26]:
!ls "drive/My Drive/SP25_mimic3"

PATIENTS.csv


In [28]:
from dataprofiler import Data, Profiler

csv_data = pd.read_csv("drive/MyDrive/SP25_mimic3/PATIENTS.csv")



In [31]:
pip install -U ydata-profiling


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.9/390.9 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 687.8/687.8 kB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.8/104.8 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 62.5 MB/s eta 0:00:00
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27081 sha256=eb076ac1d742021bbe8ae49955e2473eab1746684131fcc3ed037c03e6b0b041
  Stored in directory: /root/.cache/pip/wheels/8d/55/1a/19cd535375ed1ede0c996405ebffe34b196d78e2d9545723a2
Successfully built htmlmin


In [32]:
from ydata_profiling import ProfileReport
profile = ProfileReport(csv_data, title="Profiling Report for Patiets")

In [33]:
profile.to_file("drive/MyDrive/SP25_mimic3/PATIENTS.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]